In [4]:
import numpy as np
import pandas as pd
import seaborn as sns

In [6]:
x = pd.read_csv('/Users/okayush/Downloads/dataset and all/training_set_features.csv')
xtest=pd.read_csv('/Users/okayush/Downloads/dataset and all/test_set_features.csv')



In [8]:
x.isnull().mean()*100

respondent_id                   0.000000
xyz_concern                     0.344479
xyz_knowledge                   0.434343
behavioral_antiviral_meds       0.265848
behavioral_avoidance            0.778822
behavioral_face_mask            0.071142
behavioral_wash_hands           0.157262
behavioral_large_gatherings     0.325757
behavioral_outside_home         0.307036
behavioral_touch_face           0.479275
doctor_recc_xyz                 8.087767
doctor_recc_seasonal            8.087767
chronic_med_condition           3.635751
child_under_6_months            3.070356
health_worker                   3.010447
health_insurance               45.957989
opinion_xyz_vacc_effective      1.464036
opinion_xyz_risk                1.452803
opinion_xyz_sick_from_vacc      1.479013
opinion_seas_vacc_effective     1.729884
opinion_seas_risk               1.924589
opinion_seas_sick_from_vacc     2.010709
age_group                       0.000000
education                       5.268282
race            

In [83]:
xtest.shape, x.shape

((26708, 25), (26707, 25))

In [12]:
xtest.isnull().mean()

respondent_id                  0.000000
xyz_concern                    0.003183
xyz_knowledge                  0.004568
behavioral_antiviral_meds      0.002958
behavioral_avoidance           0.007975
behavioral_face_mask           0.000711
behavioral_wash_hands          0.001498
behavioral_large_gatherings    0.002696
behavioral_outside_home        0.003070
behavioral_touch_face          0.004793
doctor_recc_xyz                0.080875
doctor_recc_seasonal           0.080875
chronic_med_condition          0.034896
child_under_6_months           0.030440
health_worker                  0.029542
health_insurance               0.457840
opinion_xyz_vacc_effective     0.014902
opinion_xyz_risk               0.014228
opinion_xyz_sick_from_vacc     0.014041
opinion_seas_vacc_effective    0.016924
opinion_seas_risk              0.018684
opinion_seas_sick_from_vacc    0.019507
age_group                      0.000000
education                      0.052681
race                           0.000000


# preprocessing of both trainig and testing data

In [14]:
x.drop(['employment_industry','employment_occupation','hhs_geo_region','census_msa','health_insurance','education',
              'income_poverty','marital_status','rent_or_own','employment_status',],axis=1,inplace = True)
xtest.drop(['employment_industry','employment_occupation','hhs_geo_region','census_msa','health_insurance','education',
              'income_poverty','marital_status','rent_or_own','employment_status',],axis=1,inplace = True)

In [16]:
xtest.drop(['respondent_id'],axis=1,inplace = True)

In [18]:
x.drop(['respondent_id'],axis=1,inplace = True)

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import OrdinalEncoder

In [22]:
ordinal_order = ['18 - 34 Years','35 - 44 Years','45 - 54 Years','55 - 64 Years','65+ Years']

# Initialize ColumnTransformer 
preprocessor = ColumnTransformer(
    transformers=[
        ('freq_imputer', SimpleImputer(strategy='most_frequent'), [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,23,24]),  # Impute ordinal and nominal columns with most frequent value
        ('ordinal_encoder', OrdinalEncoder(categories=[ordinal_order]), ['age_group']),  # Ordinal encoding for ordinal column
        ('onehot_encoder', OneHotEncoder(drop='first',sparse=False,dtype=np.int32), ['race','sex'])  # One-hot encoding for nominal column
    ],
    remainder='passthrough')

In [24]:
x_new = preprocessor.fit_transform(x)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [26]:
xtest_new = preprocessor.transform(xtest)

In [28]:
xtest_new.shape

(26708, 27)

In [30]:
xtest_new = pd.DataFrame(xtest_new)

In [32]:
x_new = pd.DataFrame(x_new)

In [60]:
xtest_new.shape



(26708, 27)

In [36]:
# importing ytrain data
ytrain=pd.read_csv('/Users/okayush/Downloads/dataset and all/training_set_labels.csv')

In [38]:
ytrain.head()

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [79]:
submission =  ytrain.copy()

In [42]:
ytrain.drop('respondent_id',axis=1,inplace = True)

In [46]:
y1 = ytrain.iloc[:,0]
y2 = ytrain.iloc[:,1]


In [48]:
y1.head()

0    0
1    0
2    0
3    0
4    0
Name: xyz_vaccine, dtype: int64

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
import matplotlib.pyplot as plt

## for xyz_vaccine

In [147]:
model = LogisticRegression(max_iter=130)
model.fit(x_new, y1) #fitting x train for xyz vaccine

LogisticRegression(max_iter=130)

### prediciting the probability that the person will take xyz_vaccine
y_prob1 = model.predict_proba(xtest_new)[:, 1] 

In [150]:
# probability for xyz vaccine
y_prob1

array([0.08620001, 0.05952779, 0.40786389, ..., 0.13505793, 0.0532539 ,
       0.57500941])

In [152]:

y_pred = model.predict(xtest_new)

## for seasonal vaccine

In [155]:
model = LogisticRegression()
model.fit(x_new, y2)

LogisticRegression()

In [157]:
# probability for seasonal vaccine
y_prob2 = model.predict_proba(xtest_new)[:, 1]

In [159]:
y_prob2

array([0.22203307, 0.07981895, 0.68058805, ..., 0.2133906 , 0.38330472,
       0.5827557 ])

# creating the final csv file


In [162]:
# importing the format of submission file
submission = pd.read_csv('/Users/okayush/Downloads/dataset and all/submission_format.csv')

In [91]:
submission.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.5,0.7
1,26708,0.5,0.7
2,26709,0.5,0.7
3,26710,0.5,0.7
4,26711,0.5,0.7


In [93]:
submission.shape

(26708, 3)

In [164]:
submission['h1n1_vaccine'] = y_prob1

In [166]:
submission['seasonal_vaccine'] = y_prob2

In [168]:
submission.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.086200,0.222033
1,26708,0.059528,0.079819
2,26709,0.407864,0.680588
3,26710,0.478221,0.869384
4,26711,0.200640,0.479628


In [103]:
submission.to_csv('/Users/okayush/Downloads/dataset and all/submission.csv')